# Responsible AI: XAI GenAI project

## 0. Background



Based on the previous lessons on explainability, post-hoc methods are used to explain the model, such as saliency map, SmoothGrad, LRP, LIME, and SHAP. Take LRP (Layer Wise Relevance Propagation) as an example; it highlights the most relevant pixels to obtain a prediction of the class "cat" by backpropagating the relevance. (image source: [Montavon et. al (2016)](https://giorgiomorales.github.io/Layer-wise-Relevance-Propagation-in-Pytorch/))

<!-- %%[markdown] -->
![LRP example](images/catLRP.jpg)

Another example is about text sentiment classification, here we show a case of visualizing the importance of words given the prediction of 'positive':

![text example](images/textGradL2.png)

where the words highlight with darker colours indicate to be more critical in predicting the sentence to be 'positive' in sentiment.
More examples could be found [here](http://34.160.227.66/?models=sst2-tiny&dataset=sst_dev&hidden_modules=Explanations_Attention&layout=default).

Both cases above require the class or the prediction of the model. But:

***How do you explain a model that does not predict but generates?***

In this project, we will work on explaining the generative model based on the dependency between words. We will first look at a simple example, and using Point-wise Mutual Information (PMI) to compute the saliency map of the sentence. After that we will contruct the expereiment step by step, followed by exercises and questions.


## 1. A simple example to start with
Given a sample sentence: 
> *Tokyo is the capital city of Japan.* 

We are going to explain this sentence by finding the dependency using a saliency map between words.
The dependency of two words in the sentence could be measured by [Point-wise mutual information (PMI)](https://en.wikipedia.org/wiki/Pointwise_mutual_information): 


Mask two words out, e.g. 
> \[MASK-1\] is the captial city of \[MASK-2\].


Ask the generative model to fill in the sentence 10 times, and we have:

| MASK-1      | MASK-2 |
| ----------- | ----------- |
|    tokyo   |     japan   |
|  paris  |     france    |
|  london  |     england    |
|  paris  |     france    |
|  beijing |  china |
|    tokyo   |     japan   |
|  paris  |     france    |
|  paris  |     france    |
|  london  |     england    |
|  beijing |  china |

PMI is calculated by: 

$PMI(x,y)=log_2⁡ \frac{p(\{x,y\}| s-\{x,y\})}{P(\{x\}|s-\{x,y\})P(\{y\}|s-\{x,y\})}$

where $x$, $y$ represents the words that we masked out, $s$ represents the setence, and $s-\{x,y\}$ represents the sentences tokens after removing the words $x$ and $y$.

In this example we have $PMI(Tokyo, capital) = log_2 \frac{0.2}{0.2 * 0.2} = 2.32$

Select an interesting word in the sentences; we can now compute the PMI between all other words and the chosen word using the generative model:
(Here, we use a longer sentence and run 20 responses per word.)
![](images/resPMI.png)


## 2. Preparation
### 2.1 Conda enviroment

```
conda env create -f environment.yml
conda activate xai_llm
```


### 2.2 Download the offline LLM

We use the offline LLM model from hugging face. It's approximately 5 GB.
Download it using the comman below, and save it under `./models/`.
```
huggingface-cli download TheBloke/openchat-3.5-0106-GGUF openchat-3.5-0106.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False
# credit to https://huggingface.co/TheBloke/openchat-3.5-0106-GGUF
```

## 3. Mask the sentence and get the responses from LLM
### 3.1 Get the input sentence

**Remember to change the anchor word index when changing the input sentence.**

In [1]:
# Removed for consistency, so that every run uses the same sentence
# def get_input():
    # ideally this reads inputs from a file, now it just takes an input
    #return input("Enter a sentence: ")
    
# Cell 23 - Reset the sentence
sentence = "doctors assess symptoms to diagnose diseases"

anchor_word_idx = 0 # the index of the interested word
prompts_per_word = 20 # number of generated responses  

#sentence = get_input()
print("Sentence: ", sentence)

Sentence:  doctors assess symptoms to diagnose diseases


### 3.2 Load the model

In [2]:
from llama_cpp import Llama

from models.ChatModel import ChatModel
model_name = "openchat"
model = ChatModel(model_name)
print(f"Model: {model_name}")

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from /zhome/86/6/137189/XAI_GenAI_project-main/models/openchat-3.5-0106.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = openchat_openchat-3.5-0106
llama_model_loader: - kv   2:                       llama.context_length u32              = 8192
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_lo

Model: openchat


Using gguf chat template: {{ bos_token }}{% for message in messages %}{{ 'GPT4 Correct ' + message['role'].title() + ': ' + message['content'] + '<|end_of_turn|>'}}{% endfor %}{% if add_generation_prompt %}{{ 'GPT4 Correct Assistant:' }}{% endif %}
Using chat eos_token: <|end_of_turn|>
Using chat bos_token: <s>


### 3.3 Run the prompts and get all the responses


In [3]:
from tools.command_generator import generate_prompts, prefix_prompt
from tools.evaluate_response import get_replacements
from tqdm import tqdm

def run_prompts(model, sentence, anchor_idx, prompts_per_word=20):
    prompts = generate_prompts(sentence, anchor_idx)
    all_replacements = []
    for prompt in prompts:
        replacements = []
        for _ in tqdm(
            range(prompts_per_word),
            desc=f"Input: {prompt}",
        ):
            response = model.get_response(
                prefix_prompt(prompt),
            ).strip()
            if response:
                replacement = get_replacements(prompt, response)
                if replacement:
                    replacements.append(replacement)
        if len(replacements) > 0:
            all_replacements.append(replacements)
    return all_replacements

all_responses = run_prompts(model, sentence, anchor_word_idx, prompts_per_word)

Input: [MASK] [MASK] symptoms to diagnose diseases:   0%| | 0/20 [00:00<?, ?it/sllama_perf_context_print:        load time =     976.93 ms
llama_perf_context_print: prompt eval time =     976.70 ms /    48 tokens (   20.35 ms per token,    49.15 tokens per second)
llama_perf_context_print:        eval time =    2674.34 ms /    18 runs   (  148.57 ms per token,     6.73 tokens per second)
llama_perf_context_print:       total time =    3657.59 ms /    66 tokens
llama_perf_context_print:    graphs reused =         16
Input: [MASK] [MASK] symptoms to diagnose diseases:   5%| | 1/20 [00:03<01:09,  Llama.generate: 47 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     976.93 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    1180.76 ms /    10 runs   (  118.08 ms per token,     8.47 tokens per second)
llama_perf_contex

 Response is not valid. ['[mask]', '[mask]', 'symptoms', 'to', 'diagnose', 'diseases'] ['specific', '[mask]', 'symptoms', 'help', 'diagnose', 'diseases']


llama_perf_context_print:        load time =     976.93 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    1346.38 ms /    12 runs   (  112.20 ms per token,     8.91 tokens per second)
llama_perf_context_print:       total time =    1349.19 ms /    13 tokens
llama_perf_context_print:    graphs reused =         12
Input: [MASK] [MASK] symptoms to diagnose diseases:  40%|▍| 8/20 [00:12<00:15,  Llama.generate: 47 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     976.93 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    2189.06 ms /    20 runs   (  109.45 ms per token,     9.14 tokens per second)
llama_perf_context_print:       total time =    2193.97 ms /    21 tokens
llama_perf_context_prin

 Response is not valid. ['[mask]', '[mask]', 'symptoms', 'to', 'diagnose', 'diseases'] ['the', 'doctor', 'carefully', 'examines', 'the', 'patients', '[symptoms]', 'to', 'diagnose', 'diseases']


llama_perf_context_print:        load time =     976.93 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    1087.80 ms /    10 runs   (  108.78 ms per token,     9.19 tokens per second)
llama_perf_context_print:       total time =    1090.11 ms /    11 tokens
llama_perf_context_print:    graphs reused =          9
Input: [MASK] [MASK] symptoms to diagnose diseases:  50%|▌| 10/20 [00:15<00:14, Llama.generate: 47 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     976.93 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    3011.97 ms /    27 runs   (  111.55 ms per token,     8.96 tokens per second)
llama_perf_context_print:       total time =    3018.74 ms /    28 tokens
llama_perf_context_prin

...

 Response is not valid. ['[mask]', 'assess', 'symptoms', 'to', 'diagnose', '[mask]'] ['the', 'doctor', 'assesses', 'symptoms', 'to', 'diagnose', 'a', 'medical', 'condition']


llama_perf_context_print:        load time =     976.93 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    1140.14 ms /    10 runs   (  114.01 ms per token,     8.77 tokens per second)
llama_perf_context_print:       total time =    1142.52 ms /    11 tokens
llama_perf_context_print:    graphs reused =         10
Input: [MASK] assess symptoms to diagnose [MASK]:  75%|▊| 15/20 [00:19<00:06,  1Llama.generate: 47 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     976.93 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    1233.49 ms /    11 runs   (  112.14 ms per token,     8.92 tokens per second)
llama_perf_context_print:       total time =    1236.12 ms /    12 tokens
llama_perf_context_prin

 Response is not valid. ['[mask]', 'assess', 'symptoms', 'to', 'diagnose', '[mask]'] ['the', 'doctor', 'assesses', 'symptoms', 'to', 'diagnose', 'pneumonia']


llama_perf_context_print:        load time =     976.93 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    1082.39 ms /    10 runs   (  108.24 ms per token,     9.24 tokens per second)
llama_perf_context_print:       total time =    1084.88 ms /    11 tokens
llama_perf_context_print:    graphs reused =         10
Input: [MASK] assess symptoms to diagnose [MASK]:  90%|▉| 18/20 [00:23<00:02,  1Llama.generate: 47 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     976.93 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    1191.51 ms /    11 runs   (  108.32 ms per token,     9.23 tokens per second)
llama_perf_context_print:       total time =    1193.87 ms /    12 tokens
llama_perf_context_prin

 Response is not valid. ['[mask]', 'assess', 'symptoms', 'to', 'diagnose', '[mask]'] ['a', 'physician', 'assesses', 'symptoms', 'to', 'diagnose', 'a', 'disease']


In [4]:
# visualize responses
all_responses[:1]

[[['typically', 'doctors [mask] patients [mask]'],
  ['specific', '[mask]'],
  ['flulike', '[mask]'],
  ['flulike', '[mask]'],
  ['common', '[mask]'],
  ['flu', ''],
  ['', ''],
  ['migraine', 'migraine'],
  ['', ''],
  ['flulike', ''],
  ['', ''],
  ['specific', '[mask]'],
  ['flulike', '[mask]'],
  ['flulike', '[mask]'],
  ['flu', '[mask]'],
  ['common', ''],
  ['', ''],
  ['patients', 'often exhibit [mask]'],
  ['flu', '[mask]'],
  ['', '']]]

In [5]:
# Load responses
import json
input_file = "responses.json"
with open(input_file, "r") as f:
    all_responses = json.load(f)

### 3.4 EXERCISE: compute the PMI for each word

$PMI(x,y)=log_2⁡ \frac{p(\{x,y\}| s-\{x,y\})}{P(\{x\}|s-\{x,y\})P(\{y\}|s-\{x,y\})}$

* Compute the $P(x)$, $P(y)$ and $P(x,y)$ first and print it out.
* Compute the PMI for each word.
* Visualize the result by coloring. Tips: you might need to normalize the result first. 


In [6]:
import math
import numpy as np
from termcolor import colored

def compute_pmi(sentence, all_responses, anchor_idx):
    """Compute PMI between anchor word and each other word."""
    words = sentence.lower().split()
    anchor_word = words[anchor_idx]
    pmi_scores = {}
    
    for other_idx in range(len(words)):
        if other_idx == anchor_idx:
            continue
        
        # Get pattern index (skips anchor position)
        pattern_idx = other_idx if other_idx < anchor_idx else other_idx - 1
        if pattern_idx >= len(all_responses):
            continue
            
        responses = all_responses[pattern_idx]
        if not responses:
            continue
        
        # Extract anchor and other word replacements
        anchor_replacements = [r[0].lower() for r in responses if len(r) == 2]
        other_replacements = [r[1].lower() for r in responses if len(r) == 2]
        total = len(anchor_replacements)
        
        # Calculate probabilities
        count_x = sum(w == anchor_word for w in anchor_replacements)
        count_y = sum(w == words[other_idx] for w in other_replacements)
        count_xy = sum(anchor_replacements[i] == anchor_word and 
                      other_replacements[i] == words[other_idx] 
                      for i in range(total))
        
        P_x = count_x / total
        P_y = count_y / total
        P_xy = count_xy / total
        
        # Calculate PMI
        if P_x > 0 and P_y > 0 and P_xy > 0:
            pmi = math.log2(P_xy / (P_x * P_y))
        else:
            pmi = float('-inf')
        
        pmi_scores[other_idx] = {'word': words[other_idx], 'pmi': pmi, 
                                  'P_x': P_x, 'P_y': P_y, 'P_xy': P_xy}
    
    return pmi_scores

def visualize_pmi(sentence, pmi_scores, anchor_idx):
    """Visualize PMI with colored words."""
    words = sentence.split()
    
    # Normalize PMI values
    valid_pmis = [s['pmi'] for s in pmi_scores.values() if s['pmi'] != float('-inf')]
    if not valid_pmis:
        print("No valid PMI scores")
        return
    
    min_pmi, max_pmi = min(valid_pmis), max(valid_pmis)
    pmi_range = max_pmi - min_pmi if max_pmi != min_pmi else 1
    
    # Color each word
    colored_words = []
    for i, word in enumerate(words):
        if i == anchor_idx:
            colored_words.append(colored(word, 'cyan', attrs=['bold']))
        elif i in pmi_scores:
            pmi = pmi_scores[i]['pmi']
            if pmi != float('-inf'):
                norm = (pmi - min_pmi) / pmi_range
                color = 'green' if norm > 0.66 else 'yellow' if norm > 0.33 else 'red'
                colored_words.append(colored(f"{word}({pmi:.2f})", color))
            else:
                colored_words.append(word)
        else:
            colored_words.append(word)
    
    print("\n" + " ".join(colored_words) + "\n")

In [7]:
# Compute PMI scores
pmi_scores = compute_pmi(sentence, all_responses, anchor_word_idx)

# Print results
words = sentence.lower().split()
print(f"Anchor word: '{words[anchor_word_idx]}'\n")
for idx in sorted(pmi_scores.keys()):
    data = pmi_scores[idx]
    print(f"{data['word']:<15} PMI={data['pmi']:7.3f}  "
          f"P(x)={data['P_x']:.3f} P(y)={data['P_y']:.3f} P(xy)={data['P_xy']:.3f}")

# Visualize with colors
visualize_pmi(sentence, pmi_scores, anchor_word_idx)

Anchor word: 'doctors'

assess          PMI=   -inf  P(x)=0.000 P(y)=0.000 P(xy)=0.000
symptoms        PMI=  1.737  P(x)=0.300 P(y)=0.200 P(xy)=0.200
to              PMI=  1.152  P(x)=0.100 P(y)=0.450 P(xy)=0.100
diagnose        PMI=  0.322  P(x)=0.100 P(y)=0.800 P(xy)=0.100
diseases        PMI=  0.737  P(x)=0.200 P(y)=0.300 P(xy)=0.100

doctors assess symptoms(1.74) to(1.15) diagnose(0.32) diseases(0.74)



### PMI Results Interpretation (Higher PMI = stronger association)

#### Results for "doctors assess symptoms to diagnose diseases"
- The visualization shows that "symptoms" has the strongest semantic bond with "doctors" in this sentence

**High PMI: Strong Dependency**
- **symptoms (1.74)**: had the highest association with "doctors", this means that when both are amsked the model frequently generates them to fill the masked words.

**Medium PMI: Moderate Dependency**  
- **to (1.15)**: Moderate association.

**Low PMI: Weak Dependency**
- **diseases (0.74)**: Predictable from context but not uniquely tied to "doctors"
- **diagnose (0.32)**: Despite being less associated with the word doctor (pmi=0.322), it is very predictable (P(y)=0.80). This means that the word itself ("diagnose") is very frequent but paired with alternatives to "doctors" (physicians, clinicians)

**Negative PMI: No Dependency**
- **assess (-inf)**: this means the model never generated "doctors" when both were masked.


## 4. EXERCISE: Try more examples; maybe come up with your own. Report the results.

* Try to come up with more examples and, change the anchor word/number of responses, and observe the results. What does the explanation mean? Do you think it's a nice explanation? Why and why not? 
* What's the limitation of the current method? When does the method fail to explain? 

In [8]:
# EXERCISE 4

from tqdm import tqdm
import numpy as np
from IPython.display import display
from tools.command_generator import generate_prompts, prefix_prompt
from tools.evaluate_response import get_replacements

# PROMPT SAMPLING
def run_prompts(model, sentence, anchor_idx, prompts_per_word=20):
    prompts = generate_prompts(sentence, anchor_idx)
    all_replacements = []

    for prompt in prompts:
        replacements = []
        for _ in tqdm(range(prompts_per_word),
                       desc=f"Processing prompt: {prompt}",
                       leave=False):

            response = model.get_response(prefix_prompt(prompt)).strip()
            if not response:
                continue

            replacement = get_replacements(prompt, response)
            if replacement:
                replacements.append(replacement)

        if replacements:
            all_replacements.append(replacements)

    return all_replacements


# VISUALIZE RAW GENERATED SENTENCES
def visualize_generated_sentences(all_responses):
    print("\nGenerated Sentences:")
    flat = []

    for group in all_responses:
        for pair in group:
            text = " ".join(pair)
            flat.append(text)
            print(" •", text)

    print(f"\nTotal generated: {len(flat)}\n")
    return flat


# RUN PMI FOR ALL ANCHOR WORDS IN ONE SENTENCE
def run_sentence_experiment(sentence, prompts_per_word=20):
    words = sentence.split()
    anchor_indices = list(range(len(words)))

    print("\n" + "#"*110)
    print(f"ANALYZING SENTENCE:\n   '{sentence}'")
    print("#"*110)

    sentence_results = {}

    for anchor_idx in anchor_indices:
        anchor_word = words[anchor_idx]

        print("\n" + "="*90)
        print(f"Anchor index: {anchor_idx}   |   Anchor word: '{anchor_word}'")
        print("="*90)

        # Generate model outputs
        all_responses = run_prompts(model, sentence, anchor_idx, prompts_per_word)

        # Show generated sentences
        visualize_generated_sentences(all_responses)

        # Compute PMI
        pmi_scores = compute_pmi(sentence, all_responses, anchor_idx)

        # Print PMI table (no color here)
        print("\nPMI Scores:")
        if len(pmi_scores) == 0:
            print("No valid PMI scores (model did not regenerate expected words).")
        else:
            for idx in sorted(pmi_scores.keys()):
                d = pmi_scores[idx]
                print(f"{d['word']:<15} PMI={d['pmi']:7.3f}   "
                      f"P(x)={d['P_x']:.3f}  P(y)={d['P_y']:.3f}  P(xy)={d['P_xy']:.3f}")

        # Save PMI so we can visualize later
        sentence_results[anchor_word] = {
            "anchor_idx": anchor_idx,
            "pmi_scores": pmi_scores
        }

    return sentence_results


# RUN EXPERIMENT ACROSS MULTIPLE SENTENCES
experiment_sentences = [
    "doctors assess symptoms to diagnose diseases",
    "artificial intelligence transforms modern industries",
    "children love sweet ice cream on warm summer days",
    "plants require sunlight and water to grow",
    "the government announced new policies to support healthcare",
]

all_sentence_results = {}

for sent in experiment_sentences:
    results = run_sentence_experiment(sent, prompts_per_word=20)
    all_sentence_results[sent] = results

print("\n=== EXERCISE 4 COMPLETE — READY FOR VISUALIZATION ===")



##############################################################################################################
ANALYZING SENTENCE:
   'doctors assess symptoms to diagnose diseases'
##############################################################################################################

Anchor index: 0   |   Anchor word: 'doctors'


Processing prompt: [MASK] [MASK] symptoms to diagnose diseases:   0%| | 0/20 [00Llama.generate: 32 prefix-match hit, remaining 16 prompt tokens to eval
llama_perf_context_print:        load time =     976.93 ms
llama_perf_context_print: prompt eval time =     514.71 ms /    16 tokens (   32.17 ms per token,    31.09 tokens per second)
llama_perf_context_print:        eval time =    1181.95 ms /    10 runs   (  118.20 ms per token,     8.46 tokens per second)
llama_perf_context_print:       total time =    1699.52 ms /    26 tokens
llama_perf_context_print:    graphs reused =          9
Processing prompt: [MASK] [MASK] symptoms to diagnose diseases:   5%| | 1/20 [00Llama.generate: 47 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     976.93 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    1245.95 ms /    10 runs

 Response is not valid. ['[mask]', '[mask]', 'symptoms', 'to', 'diagnose', 'diseases'] ['medical', 'professionals', 'use', '[symptoms]', 'to', 'diagnose', 'diseases']


llama_perf_context_print:        load time =     976.93 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    1245.10 ms /    10 runs   (  124.51 ms per token,     8.03 tokens per second)
llama_perf_context_print:       total time =    1248.16 ms /    11 tokens
llama_perf_context_print:    graphs reused =         10
Processing prompt: [MASK] [MASK] symptoms to diagnose diseases:  25%|▎| 5/20 [00Llama.generate: 47 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     976.93 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    1496.70 ms /    12 runs   (  124.73 ms per token,     8.02 tokens per second)
llama_perf_context_print:       total time =    1499.63 ms /    13 tokens
llama_perf_context_prin

 Response is not valid. ['[mask]', '[mask]', 'symptoms', 'to', 'diagnose', 'diseases'] ['mri', 'scans', 'can', 'help', 'diagnose', 'diseases', 'by', 'identifying', '[mask]', 'symptoms']


...

 Response is not valid. ['the', 'government', 'announced', 'new', 'policies', '[mask]', 'support', '[mask]'] ['the', 'government', 'announced', 'new', 'policies', 'to', 'promote', 'environmental', 'sustainability', 'and', 'strengthen', 'national', 'security']


llama_perf_context_print:        load time =     976.93 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    1285.56 ms /    11 runs   (  116.87 ms per token,     8.56 tokens per second)
llama_perf_context_print:       total time =    1287.99 ms /    12 tokens
llama_perf_context_print:    graphs reused =         11
Processing prompt: the government announced new policies [MASK] support [MASK]: Llama.generate: 48 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     976.93 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    1305.77 ms /    11 runs   (  118.71 ms per token,     8.42 tokens per second)
llama_perf_context_print:       total time =    1308.16 ms /    12 tokens
llama_perf_context_prin

 Response is not valid. ['the', 'government', 'announced', 'new', 'policies', '[mask]', 'support', '[mask]'] ['the', 'government', 'announced', 'new', 'policies', 'to', 'increase', '[mask]', 'and', 'enhance', '[mask]']


llama_perf_context_print:        load time =     976.93 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    1624.09 ms /    14 runs   (  116.01 ms per token,     8.62 tokens per second)
llama_perf_context_print:       total time =    1627.55 ms /    15 tokens
llama_perf_context_print:    graphs reused =         14
Processing prompt: the government announced new policies [MASK] support [MASK]: Llama.generate: 48 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     976.93 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    1509.05 ms /    13 runs   (  116.08 ms per token,     8.61 tokens per second)
llama_perf_context_print:       total time =    1511.97 ms /    14 tokens
llama_perf_context_prin


Generated Sentences:
 • the japanese small businesses
 • the federal small businesses
 • the small businesses
 • the the environment
 • the french farmers
 • the small businesses
 • the australian small businesses
 • the british small businesses
 • the small businesses
 • the united states small businesses
 • the federal small businesses
 • the businesses
 • the mexican small businesses
 • the the economy
 • the british businesses
 • the japanese small businesses
 • the mexican small businesses
 • the agriculture
 • the irish farmers
 • the small businesses
 • government businesses
 • government small businesses
 • government businesses
 • government small businesses
 • government small businesses
 • government education
 • government small businesses
 • government small businesses
 • government the economy
 • government small businesses
 • government citizens
 • government small businesses
 • government farmers
 • government small businesses
 • government education
 • government busi

In [9]:
# VISUALIZATION FOR  SENTENCES

for sentence, anchor_dict in all_sentence_results.items():

    print("\n" + "#"*120)
    print(f"VISUALIZATIONS FOR SENTENCE:\n  '{sentence}'")
    print("#"*120 + "\n")

    for anchor_word, info in anchor_dict.items():
        anchor_idx = info["anchor_idx"]
        pmi_scores = info["pmi_scores"]

        print("\n" + "="*100)
        print(f"Anchor word: '{anchor_word}' (index {anchor_idx})")
        print("="*100 + "\n")

        visualize_pmi(sentence, pmi_scores, anchor_idx)
        print("\n")



########################################################################################################################
VISUALIZATIONS FOR SENTENCE:
  'doctors assess symptoms to diagnose diseases'
########################################################################################################################


Anchor word: 'doctors' (index 0)


doctors assess symptoms(0.74) to(1.15) diagnose(0.15) diseases(0.84)




Anchor word: 'assess' (index 1)

No valid PMI scores



Anchor word: 'symptoms' (index 2)


doctors assess symptoms to diagnose(0.00) diseases(-0.26)




Anchor word: 'to' (index 3)

No valid PMI scores



Anchor word: 'diagnose' (index 4)


doctors assess symptoms to diagnose diseases(0.15)




Anchor word: 'diseases' (index 5)

No valid PMI scores



########################################################################################################################
VISUALIZATIONS FOR SENTENCE:
  'artificial intelligence transforms modern industries'
#######

## Reflection on PMI Results and Explanation Quality

When we experimented with more sentences, different anchor words, and different numbers of model responses, I observed that the PMI values changed noticeably depending on how often the model regenerated particular word pairs. Words with clear semantic connections, such as doctor and diseases or ice and cream, tended to show higher PMI, while function words like to, and, or new often produced no valid PMI because the model did not reproduce them consistently. Increasing the number of responses made the PMI estimates more stable and less random.

The explanation behind PMI is that it reflects how strongly the model associates two words by comparing the probability of generating them together versus independently. A high PMI therefore indicates that the model repeatedly regenerates those words in relation to each other, revealing an underlying learned association.

Overall, PMI gives a simple and intuitive explanation because it highlights which words the model considers related. However, it is also limited, since many anchor words do not produce valid PMI, the results are sensitive to sampling noise, and the method does not explain the model’s internal reasoning processes. PMI is therefore helpful for intuition but should not be viewed as a complete explanation of model behavior.

## Limitations of the PMI-Based Explanation Method

While PMI gives a simple way to estimate dependencies between words using a generative LLM, the method comes with several important limitations:

**Exact word matching is too rigid**

The method only counts a match if the model outputs the exact same word.
But LLMs often generate synonyms or variations (e.g., kids vs children), which leads to underestimating true semantic relationships.

**Naïve tokenization creates noise**

Because the current implementation uses a simple .split(), the method struggles with contractions, punctuation, hyphenated words, and multi word expressions like ice cream or New York.
This reduces the accuracy of the PMI associations.

**Low sample size leads to unstable probabilities**

With around 20 generated completions per masked pair, estimates of P(x), P(y), and P(x,y) can be noisy.
A few lucky or unlucky generations can shift PMI ranks significantly.

**PMI only captures pairwise relationships**

Natural language meaning is often determined by interactions between several words or phrases.
PMI cannot model multi word dependencies, syntax, or context beyond two word associations.

**LLM biases influence the results**

PMI reflects the model’s training distribution and biases.
High PMI may reflect frequency biases in training data rather than genuine dependency in the sentence.

Overall, PMI gives a simple and interpretable approximation of word dependencies, but its accuracy is limited by tokenization, sampling noise, synonym variation, masking artifacts, and the behavioral nature of the method.

## 5. Bonus Exercises
### 5.1 Language pre-processing. 
In this exercise, we only lower the letters and split sentences into words; there's much more to do to pre-process the language. For example, contractions (*I'll*, *She's*, *world's*), suffix and prefix, compound words (*hard-working*). It's called word tokenization in NLP, and there are some Python packages that can do such work for us, e.g. [*TextBlob*](https://textblob.readthedocs.io/en/dev/). 


### 5.2 Better word matching
In the above example of
> Tokyo is the capital of Japan and a popular metropolis in the world.

GenAI never gives the specific word 'metropolis' when masking it out; instead, sometimes it provides words like 'city', which is not the same word but has a similar meaning. Instead of measuring the exact matching of certain words (i.e. 0 or 1), we can also measure the similarity of two words, e.g. the cosine similarity in word embedding, which ranges from 0 to 1. 